In [2]:
#everthing is found except the Name is not suppporting
import cv2
import pytesseract
import re

# Load the image using OpenCV
image_path = 'cuenta_dni_assets/cuenta_dni.jpeg'
image = cv2.imread(image_path)

# Convert the image to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Use pytesseract to extract text from the grayscale image
extracted_text = pytesseract.image_to_string(gray_image)

print(extracted_text)

lines = extracted_text.strip().split('\n')
line_8 = lines[7]

bank = 'Cuenta Dni'

# Define a regular expression pattern to match a date format like "27 de marzo 2024"
date_pattern = r'\b\d{1,2}/\d{1,2}/\d{4}\b'
amount_pattern = r'\$\s*\d[\d,\.]*'
proof_number_pattern = r'\b\d{6}\b' #have to ask martin about this

# payer_name_pattern = r'(?:\n)?([A-Z][a-z]+(?: [A-Z][a-z]+)*)'

payer_name_pattern = line_8 #have to ask martin one more img

cuit_pattern =  r'\b\d{11}\b' #MARTIN SAID Cuit IS FILLED MANUALLY 

# Find all occurrences of dates, amounts, and CUIT numbers in the extracted text
dates_found = re.findall(date_pattern, extracted_text)
amounts_found = re.findall(amount_pattern, extracted_text)
proof_number_found = re.findall(proof_number_pattern, extracted_text)  # Use cuit_pattern

payer_name_found = re.findall(payer_name_pattern , extracted_text)
cuit_found = re.findall(cuit_pattern, extracted_text)  # Use cuit_pattern

# Extract the first date, amount, and CUIT number if any are found
date = dates_found[0] if dates_found else None
amount = amounts_found[0] if amounts_found else None
proof_number = proof_number_found[0] if proof_number_found else None

payer = payer_name_found[0] if payer_name_found else None
cuit = cuit_found[0] if cuit_found else None

# Print the extracted information
print(f"Bank: {bank}")
print(f"Date found: {date}")
print(f"Amount found: {amount}")
print(f"Payer name: {payer}")
print(f"CUIT found: {cuit}")
print(f"Proof number found: {proof_number}")


fo al

Importe

$ 26.750,00

Origen
ROMERO, ALEJANDRA DANI
CUIT/CUIL/CDI: 27321504197

Cuenta origen: CAS 71595130581

Para

STRONGTECH SRL
CUIT/CUIL/CDI: 30718071484
CBU: 0720792320000004367268

Motivo

Varios

Referencia
ALE

Codigo de referencia
029645

26/03/2024 - 12:09:40hs

IV VVVVVVV VV VV

Bank: Cuenta Dni
Date found: 26/03/2024
Amount found: $ 26.750,00
Payer name: ROMERO, ALEJANDRA DANI
CUIT found: 27321504197
Proof number found: 029645


In [11]:
#takes some files are extracted and save the data
import cv2
import pytesseract
import re
import os
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment

# Function to extract data from an image and return the extracted information
def extract_data_from_image(image_path):
    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Use pytesseract to extract text from the grayscale image
    extracted_text = pytesseract.image_to_string(gray_image)

        
    lines = extracted_text.split('\n')
    lines_8 = lines[7]

    # Define regular expression patterns for different types of data to extract
    date_pattern = r'\b\d{1,2}/\d{1,2}/\d{4}\b'
    amount_pattern = r'\$\s*\d[\d,\.]*'
    proof_number_pattern = r'\b\d{6}\b'
    
    payer_name_pattern = lines_8

    cuit_pattern =  'None' #MARTIN SAID Cuit IS FILLED MANUALLY 

    # Find all occurrences of dates, amounts, and CUIT numbers in the extracted text
    dates_found = re.findall(date_pattern, extracted_text)
    amounts_found = re.findall(amount_pattern, extracted_text)
    proof_number_found = re.findall(proof_number_pattern, extracted_text)  # Use cuit_pattern

    payer_name_found = re.findall(payer_name_pattern , extracted_text)
    cuit_found = re.findall(cuit_pattern, extracted_text)  # Use cuit_pattern

    # Extract the first date, amount, and CUIT number if any are found
    date = dates_found[0] if dates_found else None
    amount = amounts_found[0] if amounts_found else None
    proof_number = proof_number_found[0] if proof_number_found else None
    
    payer = payer_name_found[0] if payer_name_found else None
    cuit = cuit_found[0] if cuit_found else None

    # Return the extracted information as a dictionary
    return {
        'BANCO': 'cuenta_dni',
        'FECHA': date,
        'IMPORTE': amount,
        'TITULAR': payer,
        'CUIT': cuit,
        'NRO COMPROBANTE': proof_number
    }

# Create a new Excel workbook
wb = Workbook()
ws = wb.active
ws.title = "Extracted Data"

# Add headers to the Excel sheet
headers = [ 'FECHA', 'IMPORTE', 'NRO COMPROBANTE', 'TITULAR', 'CUIT', 'BANCO']
ws.append(headers)

# Apply bold style to the header cells
bold_font = Font(bold=True)
for cell in ws[1]:
    cell.font = bold_font

# Apply padding to all cells
for row in ws.iter_rows():
    for cell in row:
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

# Define the folder containing the images
folder_path = 'cuenta_dni_assets'

# Process each image in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
        # Extract data from the image
        image_path = os.path.join(folder_path, filename)
        extracted_data = extract_data_from_image(image_path)

        # Add the extracted data to the Excel sheet
        row_data = [extracted_data[header] for header in headers]
        ws.append(row_data)

# Automatically adjust column widths based on content
for column_cells in ws.columns:
    max_length = 0
    for cell in column_cells:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2) * 1.2  # Adjust the multiplier as needed
    ws.column_dimensions[column_cells[0].column_letter].width = adjusted_width

# Save the Excel file
wb.save('cuenta_dni_extracted_info.xlsx')

